In [1]:
# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import sqlite3
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook_connected"


In [2]:
# %%
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

BASE_URL = "https://www12.senado.leg.br/noticias/ultimas"
DATABASE_NAME = "internet_governance_news.db"

print("Scraper Senado Notícias pronto!")


Scraper Senado Notícias pronto!


In [3]:
# %%
def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Banco e tabela prontos")

create_database()


✅ Banco e tabela prontos


In [4]:
# %%
def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


In [5]:
# %%
def montar_url(pagina):
    if pagina == 1:
        return BASE_URL
    return f"{BASE_URL}/{pagina}"


In [ ]:
# %%
noticias = []
TOTAL_PAGES = 2719
inserted = 0

for pagina in range(TOTAL_PAGES, 0, -1):
    url = montar_url(pagina)
    print(f"Coletando página {pagina}: {url}")

    r = requests.get(url, headers=HEADERS, timeout=15)
    if r.status_code != 200:
        continue

    soup = BeautifulSoup(r.text, "html.parser")
    lista = soup.find("ol", class_="lista-resultados")
    if not lista:
        continue

    itens = lista.find_all("li", recursive=False)
    print(f"  {len(itens)} notícias encontradas")

    for item in itens:
        titulo_tag = item.find("span", class_="eta")
        link_tag = item.find("a", href=True)
        data_tag = item.select_one("div.text-muted.normalis")

        if not titulo_tag or not link_tag:
            continue

        link = link_tag["href"]
        if link.startswith("/"):
            link = "https://www12.senado.leg.br" + link

        titulo = titulo_tag.get_text(strip=True)
        data = data_tag.get_text(strip=True) if data_tag else "NA"

        if insert_article(
            title=titulo,
            date=data,
            author="Agência Senado",
            url=link,
            source="Agência Senado"
        ):
            inserted += 1

        noticias.append({
            "titulo": titulo,
            "data": data,
            "link": link,
            "pagina": pagina
        })

    time.sleep(1)

print(f"\n📥 Total de novas notícias inseridas no banco: {inserted}")


Coletando página 5: https://www12.senado.leg.br/noticias/ultimas/5
  20 notícias encontradas
Coletando página 4: https://www12.senado.leg.br/noticias/ultimas/4
  20 notícias encontradas
Coletando página 3: https://www12.senado.leg.br/noticias/ultimas/3
  20 notícias encontradas
Coletando página 2: https://www12.senado.leg.br/noticias/ultimas/2
  20 notícias encontradas
Coletando página 1: https://www12.senado.leg.br/noticias/ultimas
  20 notícias encontradas

📥 Total de novas notícias inseridas no banco: 2


In [17]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT * FROM articles
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles()
print(f"📦 Total no banco: {len(df_db)} registros")
display(df_db.head(20))


📦 Total no banco: 3523 registros


,id,title,date,author,url,source
0,3578,Câmara divulga Relatório de Participação Popul...,31/12/2024 09:49,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
1,3510,Câmara divulga resultado de concurso de vídeos...,31/10/2024 11:01,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
2,3785,Congresso se ilumina na cor laranja em apoio à...,31/07/2025 15:07,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
3,3389,Projeto sobre aborto é o destaque da participa...,31/07/2024 16:57,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
4,3390,Programa da Câmara voltado ao público infantoj...,31/07/2024 11:59,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
5,4071,Dúvidas sobre as eleições,31/07/2006 00h00,Agência Senado,https://www12.senado.leg.br/noticias/especiais...,Agência Senado
6,4009,Tabagismo,31/05/2004 00h00,Agência Senado,https://www12.senado.leg.br/noticias/especiais...,Agência Senado
7,3644,"Autor da mostra ""A cidade se perde nas ausênci...",31/03/2025 09:08,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
8,3194,Visitação institucional ao Congresso fica susp...,31/01/2024 15:04,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados
9,3195,Medalha do Mérito Legislativo de 2023 será ent...,31/01/2024 11:50,Agência Câmara,https://www.camara.leg.br/assessoria-de-impren...,Câmara dos Deputados


In [ ]:
def plot_charts(df):
    if df.empty:
        print("❌ Sem dados para gráficos")
        return
    
    top15 = df.head(15).copy()
    top15['rank'] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x='rank',
        y='title',
        orientation='h',
        title='Top 15 Notícias – Internet Governance'
    )
    fig1.update_layout(height=600)
    fig1.show()

    source_count = df["source"].value_counts().reset_index()
    source_count.columns = ["source", "count"]

    fig2 = px.pie(
        source_count,
        names="source",
        values="count",
        title="Distribuição por Fonte"
    )
    fig2.show()


    text = ' '.join(df['title'].astype(str)).lower()
    words = re.findall(r'\b\w{4,}\b', text)

    wc = (
        pd.Series(words)
        .value_counts()
        .head(20)
        .reset_index()
    )
    wc.columns = ['palavra', 'freq']

    fig3 = px.treemap(
        wc,
        path=['palavra'],
        values='freq',
        title='Nuvem de Palavras – Títulos'
    )
    fig3.show()


In [18]:
# %%
plot_charts_db(df_db)